# query suggestion

## 产品形态

<img src="img/csug.png" width = "800" height = "600" alt="中文提示词" align=left />

<img src="img/csug_pinyin.png" width = "800" height = "600" alt="拼音提示词" align=right />

中文的搜索提示通常有两种形式，一种是全中文的提示，另外一种是拼音的提示如上图2.以输入`图`为例，在输入后，下方的提示框回列出10个候选项，工用户选择，当然作为一名技术人员，我对图片，特别美女图片可能没那么大兴趣，我意图是搜索一下“图计算”的资料，那么这个提示词的结果对我而言就不是很友好的结果，但是这并不能作为一个评估标准，判定google的sug结果不好，如果我是对图片比较感兴趣，那这个提示就是很好，在第二，第三位，很精准的提示了我的意图，无论是那种情况，就以本次使用的情况作为判断，这个判断就是很主观的结论。那问题来了，他从哪里来，他到哪里去呢。
也就是：
* 提示词的来源是哪里
* 提示词结果如何评估
下面依次展开讨论

## 搜索提示

query suggestion，也就是我们常见的下拉提示框，也有另外一种称为“联想词“，整体范畴是搜索的范畴。也是推荐的一种，在公司内，在nlp或者推荐团队。本质上， sug业务是根据用户的当前输入，自动提供若干query供用户选择。

在学术上，他有另外一个名字 QAC -- query autocompletion


这些query的来源大多是query log中挖掘出来大量的候选query，另一部是生成式query，在冷启动、关联域挖掘等场景应用。无论是挖掘的用户query还是引导的生成query，检索时，都要求前缀相同（在数据不足时也可能是语义相同，中缀，后缀相同等情况）。通常情况下，依次检索会产生大量的query候选，如何筛选这些query？我们还需要一个排序规则，对成百上千的query进行排序，最终返回给用户一个top10的列表。

## sug的作用

在开始具体的技术之前，我们先要搞清楚为什么做sug。从宏观角度，做sug是为了搜索导流，为了让这个宏观的目标能够以正确的姿势落地，进一步细化sug系统的作用：

* 明确用户搜索意图 当用户输入一个，有可能不是一个标准的提问，通过query sug提示，能够帮助用户用正确的方式提问，避免用户输入的query是长尾的query，进而带来召回不足、排序过低等情况。

* 提升产品体验，方便用户输入query对 某些用户来说，如果候选的query供用户选择，能够大大降低用户的输入成本。
* 提搞搜索的精度 在query sug中的query，通常理解上时我们的热门query和精品内容。可以保证召回，相关性，时效性等。如果这些query中出现召回不足，或者无自然结果的情况，技术、运营层面需要做更多的工作。 
* 缩短搜索总时长
* 提升用户体验

## 技术需求

### 低延迟

通常搜索系统的整体延迟控制在1s之内，基本可以满足用户的要求，但实际上，sug是一个特殊的场景，对性能的要求有特别的要求。
用户在输入一个字之后，前端检测到用户已经输入，就需要向后台请求sug列表，前端渲染展示的时间，要在用户输入下一个字的时间内展示出来，如果展示出来的时间过长，用户就体验就回收很大的影响。通常，sug的要求在20-100ms的区间内返回结果。这其中，不仅仅是工程需要部署相应的IDC解决全国用户访问因为网络链路带来的延迟，在算法选择上，也要保障是轻量级排序、召回、rerank机制。

### 高吞吐

在保证低延迟、低响应时间的同时，sug业务还要满足高QPS的业务要求。多机房，负载均衡、限流等策略在后面的工程方向解决方案中阐述。

## 技术策略

通常情况下，有下面几种策略，在实际业务中，通常是下面几种方式综合使用。

### 基于全量日志

这种方式是将全部用户的全部检索query，按照预定的几个目标进行融合排序。比如在视频搜索的场景，按照query的pv，ctr，播放时长做融合公式：

$score(q)=\alpha\times pv + \beta \times ctr + \gamma \times playtime \tag{1}$ 

将上述feature输入到模型中，拟合出最终得分。依据得分进行排序。

### 基于上下文的自动补全

依赖search session。这里有一个假设，假设用户同一个session的多次搜索，是相关的。将同一个session中的query映射到同一个空间，计算初始query之间的相似度，分数越高的query，越能表征用户的意图，比如搜索篮球鞋，耐克篮球鞋，阿迪篮球等，都有较高的相关性。

### 基于用户信息的自动补全

通过识别用户的行为，集合推荐的用户profile，识别用的意图，对用户的行为进行建模，进而补全。

### 选型比较

| |基于日志补全|基于用户信息补全|基于上下文补全|
|----|----|----|----|
|优势|简单|精准|精准|
|劣势|长尾无解|相关性低|新兴趣发现困难|
|要求|query频率不变|query之间有相关性|query查询要有兴趣|

上表展示了各种补全方式的优势和劣势，在实际应用中，根据具体业务的特点，需要做一些tradeoff，当然可以在最外层用lr模型，让模型自己去帮我做做出选择。

## 技术方案

前面巴拉巴拉一堆，那要怎么做呢，才能实现一套80分的系统呢。我们先梳理下系统的组件和模型

### 模块

* 离线挖掘
* 召回
* 排序
* 规则排序
* 业务重排
* 业务过滤

### 离线挖掘

搜索日志挖掘是最常用的一种方式，有批量和实时日志挖掘两种，批量日志挖掘的情况下，通常日志更新级别是天级别更新，实时日志是分钟级别的更新。在不同的业务场景下，有不同的需求，如资讯类，对新热点的sug要求较高，当然这背后有一些列的需求，热点发现，热点事件出发等。在不同的场景下要做不同的tradeoff。

#### query生成

业务上有大量新内容进入到系统，在没有用户的搜索日志，点击记录之前，会有冷启动的问题。在sug这个场景下，通过预生成query帮助系统进行倒流，从而解决冷启动问题。有几种方法可以实现这个功能。

##### 常规拼接以及文章关键字提取

第一种方式比较常规，将doc的title，author，提取文章关键词，将这些内容拼接起来，作为生成query列表
如一片文章是关于库里的，标题是“评价库里历史地位”文章类别是体育，球队域是勇士，文章主题提取为 三分 夺冠，可以拼接为：
```text
库里历史地位
勇士库里
库里冠军
库里三分球
```
等等
这种方式好处是实现容易，缺点是主题不够明确，精准度欠缺。

##### 生成算法

生成模型利用的是seq2seq的算法，比如机器翻译利用encoder/decoder架构，生成query，那么此处也是同理，可以利用模型，控制decoder输出，
来生成query，在比如bert模型。

无论使用上述哪种生成算法，都会带来几个问题，第一个是工程上的难点，每篇doc都会生成若干query，在召回引擎因某种原因要在线删除doc时，都会带来，单doc多记录问题，在性能要求较高的系统，需要做好权衡，高内存高性能还是低内存低性能。

### 召回

* 前缀召回 前缀召回在搜索领域内使用比较普遍，通常是用trie树完成这项功能
* term 召回 term召回是根据query，建立倒排索引。
* 向量化召回 向量召回早先试用kd完成，近年，向量召回有了长足的发展，从knn发展到ann，延迟从几十ms降低到5ms以内，项目可参见[tann](https://github.com/gottingen/titan-ann)，技术细节参阅[向量引擎](../vector_recall/index.ipynb)
* 基于图召回 本质上时基于query的随机游走，但本次系统不实现该功能

前缀召回、term召回以及向量召回，lambda引擎中，经过精心设计，巧妙聚合到一个引擎当中，细节在lambda引擎设计部分展开讨论

### 排序

#### 粗排

精排是比较复杂的排序模型，在精排之前，粗排将不靠谱的doc先过滤掉，从性能的角度考虑，通常经过静态分数，主要是是先验的指标，如pv ctr，bm25等。

//TODO 细节完善

#### 精排

常用的FFM或者是DeepFm的流式模型，与推荐模型类似
同时，与推荐模型也有很大区别，出性能的考虑，模型不能做大。
第二个是长尾问题，从全局角度统计，点击率会比较高，因此在做模型时，要经过一定的平滑，如L2正则化
在一个问题是问题是样本的选择，即使推送的query与用户的输入相同，用户可能选择自己输入。那么在做模型时，样本的选择至关重要。通常在训练时区掉
id类型的特征，增加单字的特征，再一个，进行适度的deropout，保证模型的泛华能力。

TODD 细节完善

### 规则排序

精排（模型排序）不能解决全部的问题，如运营需求，中文置顶，搜索历史置顶等需求，rerank并没有太多技术细节，主要是产品的策略和常规的规则

### 业务重排

### 评估

* 客观指标
    * 使用率
    * 点击量
    * 前三点击率
    * 用户输入长度
    * 等等
* 人工指标 偏体验
    * 内容
        * 质量-- 优质query占比
        * 安全 -- 合规、非色情等
        * 相关性 相关的比例
    * 评估方式
        * 竞品
        * top query先验